In [ ]:
import math

Sun position 

=> α=90-Φ+δ

In [3]:
def sun_position(latitude, declination):
    return 90 - latitude + declination

Where declination is calculated as

=> δ= 23.45° * Sin[ 360/365 (284+d)] 

and 284 is an adjustation value as declination not being zero on January 1

In [5]:
def declination(day):
    return 23.45 * math.sin((360/365) * 284 + day)

Energy received taking into acount the inclinationg of the surface 

=> G (inclined) = G(horizontal) * Sin (α+β) / Sinα

β is the ° of the surface

In [7]:
def radiation_with_angle_simple(raw_radiation, sun_position, panel_angle):
    return raw_radiation * math.sin(sun_position, panel_angle) / math.sin(sun_position)